In [59]:
import numpy as np
import pandas as pd
import csv

from os import listdir

import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [60]:
dir_run = '../results'

NSEEDS = 10


In [61]:
def get_best_result_df(best_result_file_path):
    history = []
    with open(best_result_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        counter = 0
        for row in reader:
            if counter == 1:
                params = row
            elif counter == 3:
                history = row
            elif counter == 5:
                configs = row
            counter += 1
    # D, AEdAO, PdD, Z, fitness
    D       = float(params[0])
    AEdAO   = float(params[1])
    PdD     = float(params[2])
    Z       = int(params[3]) if len(params[3]) == 1 else int(float(params[3]))
    fitness = float(params[6])
    # history    
    if len(history) > 0:
        history = [float(h) for h in history]
    # configs
    solver_name = configs[0]
    vs        = float(configs[1])
    seed        = int(configs[4])
    # create new entry in df
    new_data = {'D': D, 
                'AEdAO': AEdAO, 
                'PdD': PdD,
                'Z': int(Z),
                'Brake Power': -fitness,
                'Seed': seed,
                'Algorithm': solver_name,
                'VS': vs}
    return new_data

## Get best by seed

In [62]:
data = {'D': [], 'AEdAO': [], 'PdD': [], 'Z': [], 'Brake Power': [], 'Seed': [], 'Algorithm': [], 'VS': []}
df_results = pd.DataFrame(data)

for algorithm in listdir(dir_run):
    algorithm_folder = dir_run + '/' + algorithm

    for speed in listdir(algorithm_folder):  # Loop sobre as velocidades
        speed_folder = algorithm_folder + '/' + speed
        
        for seed in range(NSEEDS):
            seed_folder = speed_folder + '/' + str(seed)
                
            best_result_file = [filename for filename in listdir(seed_folder) if 'best_results' in filename]
            
            if len(best_result_file) > 0:
                best_result_file = best_result_file[0]
                best_result_file = seed_folder+'/'+best_result_file
                new_data = get_best_result_df(best_result_file)
                df_results = pd.concat([df_results, pd.DataFrame(new_data, index=[0])], ignore_index=True)
 
df_results = df_results.astype({"Z": int, "Seed": int})   
df_results = df_results.drop('Seed', axis=1)

In [ ]:
df_results

In [63]:
df_cmaes = df_results.loc[df_results['Algorithm'] == 'cmaes']
df_openaies = df_results.loc[df_results['Algorithm'] == 'openaies']
df_de = df_results.loc[df_results['Algorithm'] == 'DE']
df_de_mod = df_results.loc[df_results['Algorithm'] == 'DE_mod']

df_cmaes = df_cmaes.drop('Algorithm', axis=1)
df_openaies = df_openaies.drop('Algorithm', axis=1)
df_de = df_de.drop('Algorithm', axis=1)
df_de_mod = df_de_mod.drop('Algorithm', axis=1)

In [ ]:
df_de_mod

In [64]:
# Generate replicaed DE dataset
# Replicated DE has only 2 because it only run for 7 and 7.5 of V_S

df_de_7   = df_de.loc[df_de['VS'] == 7.0]
df_de_7_5 = df_de.loc[df_de['VS'] == 7.5]

df_de_7   = df_de_7.drop('VS', axis=1)
df_de_7_5 = df_de_7_5.drop('VS', axis=1)

In [65]:
# Generate modified DE dataset

df_de_mod_7   = df_de_mod.loc[df_de_mod['VS'] == 7.0]
df_de_mod_7_5 = df_de_mod.loc[df_de_mod['VS'] == 7.5]
df_de_mod_8   = df_de_mod.loc[df_de_mod['VS'] == 8.0]
df_de_mod_8_5 = df_de_mod.loc[df_de_mod['VS'] == 8.5]

df_de_mod_7   = df_de_mod_7.drop('VS', axis=1)
df_de_mod_7_5 = df_de_mod_7_5.drop('VS', axis=1)
df_de_mod_8   = df_de_mod_8.drop('VS', axis=1)
df_de_mod_8_5 = df_de_mod_8_5.drop('VS', axis=1)

In [66]:
# Generate cmaes dataset

df_cmaes_7      = df_cmaes.loc[df_cmaes['VS'] == 7.0]
df_cmaes_7_5    = df_cmaes.loc[df_cmaes['VS'] == 7.5]
df_cmaes_8      = df_cmaes.loc[df_cmaes['VS'] == 8.0]
df_cmaes_8_5    = df_cmaes.loc[df_cmaes['VS'] == 8.5]

df_cmaes_7      = df_cmaes_7.drop('VS', axis=1)
df_cmaes_7_5    = df_cmaes_7_5.drop('VS', axis=1)
df_cmaes_8      = df_cmaes_8.drop('VS', axis=1)
df_cmaes_8_5    = df_cmaes_8_5.drop('VS', axis=1)

In [67]:
# Generate openaies dataset

df_openaies_7   = df_openaies.loc[df_openaies['VS'] == 7.0]
df_openaies_7_5 = df_openaies.loc[df_openaies['VS'] == 7.5]
df_openaies_8   = df_openaies.loc[df_openaies['VS'] == 8.0]
df_openaies_8_5 = df_openaies.loc[df_openaies['VS'] == 8.5]

df_openaies_7   = df_openaies_7.drop('VS', axis=1)
df_openaies_7_5 = df_openaies_7_5.drop('VS', axis=1)
df_openaies_8   = df_openaies_8.drop('VS', axis=1)
df_openaies_8_5 = df_openaies_8_5.drop('VS', axis=1)

## Statistics

In [10]:
df_de_7.describe()

,D,AEdAO,PdD,Z,Brake Power
count,10.0,10.000000,10.000000,10.0,10.000000
mean,0.8,0.661627,0.669448,5.0,81.551321
std,0.0,0.028011,0.018635,0.0,0.125404
min,0.8,0.625847,0.640228,5.0,81.432608
25%,0.8,0.647030,0.658366,5.0,81.477026
50%,0.8,0.657047,0.664718,5.0,81.528028
75%,0.8,0.668007,0.678906,5.0,81.575100
max,0.8,0.730914,0.701546,5.0,81.865217


## Shapiro-Wilk Test

In [68]:
def shapiro_wilk(df):
    try:
        stat, p_value = shapiro(df['Brake Power'])
        return {'Estatística de Shapiro-Wilk': stat, 'Valor p': p_value}
    except Exception as e:
        print('Erro ' + str(e))
        return {'Erro': str(e)}

## Statistical Tests

Hypothesis H0 (Null Hypothesis): Brake Power obtained by Algorithm 1 does not have statistical difference compared to that obtained by Algorithm 2.

We will try to prove this with 95% confidence.

In the hypothesis test (which is either the t-test or Mann-Whitney depending on the distribution format), it will return a value. If the value is less than 5% (p=0.05), they are different; otherwise, there is no way to prove the difference.

In [69]:
def run_statistical_test(df_a, df_b):
    significance_level = 0.05
    
    df_a_result = shapiro_wilk(df_a)
    df_b_result = shapiro_wilk(df_b)
    
    # If p-values less than 0.05 (Significance Level) use Mann-Whitney U, otherwise, use T-Test
    if df_a_result['Valor p'] > significance_level and df_b_result['Valor p'] > significance_level:
        # Use T-Test
        t_statistic, p_value = ttest_ind(df_a['Brake Power'], df_b['Brake Power'])
        return {'Algorithm':'t-test', 'Statistic':statistic, 'p-value':p_value}
    else:
        # Use Mann-Whitney U
        statistic, p_value = mannwhitneyu(df_a['Brake Power'], df_b['Brake Power'], alternative='two-sided')
        return {'Algorithm':'mann-whitney-u', 'Statistic':statistic, 'p-value':p_value}
        


#### Combinations of tests

For V_S [7 and 7.5], we need to compare the four algorithms:

DE      - DE_MOD \
DE      - CMA-ES \
DE      - OpenAI-ES \
DE_MOD  - CMA-ES \
DE_MOD  - OpenAI-ES \
CMA-ES  - OpenAI-ES 

For V_S [8 and 8.5], replicated DE didn't run. Because of that we need to run less tests:

DE_MOD  - CMA-ES \
DE_MOD  - OpenAI-ES \
CMA-ES  - OpenAI-ES 

In [70]:
# V_S = 7
de_de_mod_7     = run_statistical_test(df_de_7, df_de_mod_7)
de_cma_7        = run_statistical_test(df_de_7, df_cmaes_7)
de_openai_7     = run_statistical_test(df_de_7, df_openaies_7)
de_mod_cma_7    = run_statistical_test(df_de_mod_7, df_cmaes_7)
de_mod_openai_7 = run_statistical_test(df_de_mod_7, df_openaies_7)
cma_openai_7    = run_statistical_test(df_cmaes_7, df_openaies_7)

In [71]:
print(de_de_mod_7)
print(de_cma_7)
print(de_openai_7)
print(de_mod_cma_7)
print(de_mod_openai_7)
print(cma_openai_7)

{'Algorithm': 'mann-whitney-u', 'Statistic': 78.0, 'p-value': 0.03763531378731424}
{'Algorithm': 'mann-whitney-u', 'Statistic': 82.0, 'p-value': 0.017257456083119765}
{'Algorithm': 'mann-whitney-u', 'Statistic': 30.0, 'p-value': 0.14046504815835495}
{'Algorithm': 'mann-whitney-u', 'Statistic': 57.0, 'p-value': 0.6231762238821174}
{'Algorithm': 'mann-whitney-u', 'Statistic': 15.0, 'p-value': 0.009108496398030963}
{'Algorithm': 'mann-whitney-u', 'Statistic': 12.0, 'p-value': 0.004586392080253494}


In [72]:
# V_S = 7_5
de_de_mod_7_5       = run_statistical_test(df_de_7_5, df_de_mod_7_5)
de_cma_7_5          = run_statistical_test(df_de_7_5, df_cmaes_7_5)
de_openai_7_5       = run_statistical_test(df_de_7_5, df_openaies_7_5)
de_mod_cma_7_5      = run_statistical_test(df_de_mod_7_5, df_cmaes_7_5)
de_mod_openai_7_5   = run_statistical_test(df_de_mod_7_5, df_openaies_7_5)
cma_openai_7_5      = run_statistical_test(df_cmaes_7_5, df_openaies_7_5)

In [73]:
print(de_de_mod_7_5)
print(de_cma_7_5)
print(de_openai_7_5)
print(de_mod_cma_7_5)
print(de_mod_openai_7_5)
print(cma_openai_7_5)

{'Algorithm': 'mann-whitney-u', 'Statistic': 84.0, 'p-value': 0.011329696684474668}
{'Algorithm': 'mann-whitney-u', 'Statistic': 85.0, 'p-value': 0.009108496398030963}
{'Algorithm': 'mann-whitney-u', 'Statistic': 28.0, 'p-value': 0.10410988966022681}
{'Algorithm': 'mann-whitney-u', 'Statistic': 48.0, 'p-value': 0.9097218891455553}
{'Algorithm': 'mann-whitney-u', 'Statistic': 1.0, 'p-value': 0.00024612812790522973}
{'Algorithm': 'mann-whitney-u', 'Statistic': 0.0, 'p-value': 0.00018267179110955002}


In [74]:
# V_S = 8
de_mod_cma_8      = run_statistical_test(df_de_mod_8, df_cmaes_8)
de_mod_openai_8   = run_statistical_test(df_de_mod_8, df_openaies_8)
cma_openai_8      = run_statistical_test(df_cmaes_8, df_openaies_8)

In [75]:
print(de_mod_cma_8)
print(de_mod_openai_8)
print(cma_openai_8)

{'Algorithm': 'mann-whitney-u', 'Statistic': 72.0, 'p-value': 0.10410988966022681}
{'Algorithm': 'mann-whitney-u', 'Statistic': 25.0, 'p-value': 0.06402210128302689}
{'Algorithm': 'mann-whitney-u', 'Statistic': 2.0, 'p-value': 0.00032983852077799353}


In [76]:
# V_S = 8.5
de_mod_cma_8_5      = run_statistical_test(df_de_mod_8_5, df_cmaes_8_5)
de_mod_openai_8_5   = run_statistical_test(df_de_mod_8_5, df_openaies_8_5)
cma_openai_8_5      = run_statistical_test(df_cmaes_8_5, df_openaies_8_5)

In [77]:
print(de_mod_cma_8_5)
print(de_mod_openai_8_5)
print(cma_openai_8_5)

{'Algorithm': 'mann-whitney-u', 'Statistic': 78.0, 'p-value': 0.03763531378731424}
{'Algorithm': 'mann-whitney-u', 'Statistic': 32.0, 'p-value': 0.18587673236587576}
{'Algorithm': 'mann-whitney-u', 'Statistic': 14.0, 'p-value': 0.0072845570094796615}


### Bonferroni correction

In [78]:
def bonferroni_correction(mw_results, alpha=0.05):
    """
    Applies Bonferroni correction to the results of the Mann-Whitney U test.

    Args:
    - mw_results (list): List containing dictionaries with the results of the Mann-Whitney U test.
                         Each dictionary should have keys 'Algorithm', 'Statistic', and 'p-value'.
    - alpha (float): Significance level.

    Returns:
    - corrected_results (list): List containing the corrected results after Bonferroni correction.
                                Each element of the list is a boolean indicating whether the null hypothesis was rejected
                                (True for rejected, False otherwise).
    """
    # Extracting p-values from mw_results
    p_values = [result['p-value'] for result in mw_results]
    
    # Applying Bonferroni correction
    corrected_p_values = multipletests(p_values, alpha=alpha, method='bonferroni')[1]
    
    # Converting corrected p-values into boolean results
    # corrected_results = [p_value < alpha for p_value in corrected_p_values]
    
    return corrected_p_values

In [79]:
# V_S = 7

results = [de_de_mod_7, de_cma_7, de_openai_7, de_mod_cma_7, de_mod_openai_7, cma_openai_7]

corrected_results = bonferroni_correction(results)

print("Results corrected after Bonferroni:")
print(corrected_results)

de_de_mod_7_corrected     = corrected_results[0]
de_cma_7_corrected        = corrected_results[1]
de_openai_7_corrected     = corrected_results[2]
de_mod_cma_7_corrected    = corrected_results[3]
de_mod_openai_7_corrected = corrected_results[4]
cma_openai_7_corrected    = corrected_results[5]

Results corrected after Bonferroni:
[0.22581188 0.10354474 0.84279029 1.         0.05465098 0.02751835]


In [80]:
# V_S = 7.5

results = [de_de_mod_7_5, de_cma_7_5, de_openai_7_5, de_mod_cma_7_5, de_mod_openai_7_5, cma_openai_7_5]

corrected_results = bonferroni_correction(results)

print("Results corrected after Bonferroni:")
print(corrected_results)

de_de_mod_7_5_corrected     = corrected_results[0]
de_cma_7_5_corrected        = corrected_results[1]
de_openai_7_5_corrected     = corrected_results[2]
de_mod_cma_7_5_corrected    = corrected_results[3]
de_mod_openai_7_5_corrected = corrected_results[4]
cma_openai_7_5_corrected    = corrected_results[5]

Results corrected after Bonferroni:
[0.06797818 0.05465098 0.62465934 1.         0.00147677 0.00109603]


In [81]:
# V_S = 8

results = [de_mod_cma_8, de_mod_openai_8, cma_openai_8]

corrected_results = bonferroni_correction(results)

print("Results corrected after Bonferroni:")
print(corrected_results)

de_mod_cma_8_corrected     = corrected_results[0]
de_mod_openai_8_corrected  = corrected_results[1]
cma_openai_8_corrected     = corrected_results[2]

Results corrected after Bonferroni:
[0.31232967 0.1920663  0.00098952]


In [82]:
# V_S = 8.5

results = [de_mod_cma_8_5, de_mod_openai_8_5, cma_openai_8_5]

corrected_results = bonferroni_correction(results)

print("Results corrected after Bonferroni:")
print(corrected_results)

de_mod_cma_8_5_corrected     = corrected_results[0]
de_mod_openai_8_5_corrected  = corrected_results[1]
cma_openai_8_5_corrected     = corrected_results[2]

Results corrected after Bonferroni:
[0.11290594 0.5576302  0.02185367]
